In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

C:\Users\Jason\AppData\Local\Temp\ipykernel_24108\3187316332.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Initial imports.
import numpy as np                          # for array operations
import pandas as pd                          # for working with DataFrames
import matplotlib.pyplot as plt        # for data visualization
%matplotlib inline                               
import seaborn as sns
from pathlib import Path
import tensorflow as tf


#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder                # transform categorical features into numerical dummy features (training)
from sklearn.model_selection import train_test_split              # for splitting the data
from sklearn.metrics import mean_squared_error                   # for calculating average
from sklearn.ensemble import RandomForestRegressor         # for building the model
from sklearn.metrics import r2_score                                        # evaluate performance of a linear regression model
#from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [3]:
# Loading data
file_path = ("causes_of_death.csv")
death_df = pd.read_csv(file_path)
death_df.head()

,Year,Cause Name 113,Cause Name,State,Deaths,Death Rate
0,2017,"Accidents (unintentional injuries) (V01-X59,Y85-Y86)",Unintentional injuries,United States,169936,49.40
1,2017,"Accidents (unintentional injuries) (V01-X59,Y85-Y86)",Unintentional injuries,Alabama,2703,53.80
2,2017,"Accidents (unintentional injuries) (V01-X59,Y85-Y86)",Unintentional injuries,Alaska,436,63.70
3,2017,"Accidents (unintentional injuries) (V01-X59,Y85-Y86)",Unintentional injuries,Arizona,4184,56.20
4,2017,"Accidents (unintentional injuries) (V01-X59,Y85-Y86)",Unintentional injuries,Arkansas,1625,51.80


In [4]:
# Generate our categorical variable list
death_cat = death_df.dtypes[death_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
death_df[death_cat].nunique()

Cause Name 113    11
Cause Name        11
State             52
dtype: int64

In [5]:
death_df.dtypes

Year                int64
Cause Name 113     object
Cause Name         object
State              object
Deaths              int64
Death Rate        float64
dtype: object

In [6]:
death_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10868 entries, 0 to 10867
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            10868 non-null  int64  
 1   Cause Name 113  10868 non-null  object 
 2   Cause Name      10868 non-null  object 
 3   State           10868 non-null  object 
 4   Deaths          10868 non-null  int64  
 5   Death Rate      10868 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 509.6+ KB


In [7]:
# Check the unique value counts to see if binning is required
death_df['Cause Name'].value_counts()

Unintentional injuries     988
All causes                 988
Alzheimer's disease        988
Stroke                     988
CLRD                       988
Diabetes                   988
Heart disease              988
Influenza and pneumonia    988
Suicide                    988
Cancer                     988
Kidney disease             988
Name: Cause Name, dtype: int64

In [8]:
# Check the unique value counts to see if binning is required
death_df['State'].value_counts()

United States           209
Alabama                 209
Nebraska                209
Nevada                  209
New Hampshire           209
New Jersey              209
New Mexico              209
New York                209
North Carolina          209
North Dakota            209
Ohio                    209
Oklahoma                209
Oregon                  209
Pennsylvania            209
Rhode Island            209
South Carolina          209
South Dakota            209
Tennessee               209
Texas                   209
Utah                    209
Vermont                 209
Virginia                209
Washington              209
West Virginia           209
Wisconsin               209
Montana                 209
Missouri                209
Mississippi             209
Hawaii                  209
Alaska                  209
Arizona                 209
Arkansas                209
California              209
Colorado                209
Connecticut             209
Delaware            

In [9]:
# Drop columns that are not useful
death_df =  death_df.drop('Cause Name 113', axis =1)

In [10]:
death_df.head()

,Year,Cause Name,State,Deaths,Death Rate
0,2017,Unintentional injuries,United States,169936,49.40
1,2017,Unintentional injuries,Alabama,2703,53.80
2,2017,Unintentional injuries,Alaska,436,63.70
3,2017,Unintentional injuries,Arizona,4184,56.20
4,2017,Unintentional injuries,Arkansas,1625,51.80


In [11]:
death_df_encoded = pd.get_dummies(death_df, columns=["Cause Name", "State"])
death_df_encoded.head()

,Year,Deaths,Death Rate,Cause Name_All causes,Cause Name_Alzheimer's disease,Cause Name_CLRD,Cause Name_Cancer,Cause Name_Diabetes,Cause Name_Heart disease,Cause Name_Influenza and pneumonia,Cause Name_Kidney disease,Cause Name_Stroke,Cause Name_Suicide,Cause Name_Unintentional injuries,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_District of Columbia,State_Florida,State_Georgia,State_Hawaii,State_Idaho,State_Illinois,State_Indiana,State_Iowa,State_Kansas,State_Kentucky,State_Louisiana,State_Maine,State_Maryland,State_Massachusetts,State_Michigan,State_Minnesota,State_Mississippi,State_Missouri,State_Montana,State_Nebraska,State_Nevada,State_New Hampshire,State_New Jersey,State_New Mexico,State_New York,State_North Carolina,State_North Dakota,State_Ohio,State_Oklahoma,State_Oregon,State_Pennsylvania,State_Rhode Island,State_South Carolina,State_South Dakota,State_Tennessee,State_Texas,State_United States,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,2017,169936,49.40,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,2017,2703,53.80,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2017,436,63.70,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2017,4184,56.20,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2017,1625,51.80,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
death_df.dtypes

Year            int64
Cause Name     object
State          object
Deaths          int64
Death Rate    float64
dtype: object

In [13]:
# Define the features data set.
X = death_df_encoded.copy()
#X = X.drop(["Death Rate","Deaths", "Cause Name_All causes", "Cause Name_Cancer", "Cause Name_Heart disease"], axis=1)
X = X.drop(["Death Rate","Deaths", "Cause Name_All causes"], axis=1)
X.head()


,Year,Cause Name_Alzheimer's disease,Cause Name_CLRD,Cause Name_Cancer,Cause Name_Diabetes,Cause Name_Heart disease,Cause Name_Influenza and pneumonia,Cause Name_Kidney disease,Cause Name_Stroke,Cause Name_Suicide,Cause Name_Unintentional injuries,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_District of Columbia,State_Florida,State_Georgia,State_Hawaii,State_Idaho,State_Illinois,State_Indiana,State_Iowa,State_Kansas,State_Kentucky,State_Louisiana,State_Maine,State_Maryland,State_Massachusetts,State_Michigan,State_Minnesota,State_Mississippi,State_Missouri,State_Montana,State_Nebraska,State_Nevada,State_New Hampshire,State_New Jersey,State_New Mexico,State_New York,State_North Carolina,State_North Dakota,State_Ohio,State_Oklahoma,State_Oregon,State_Pennsylvania,State_Rhode Island,State_South Carolina,State_South Dakota,State_Tennessee,State_Texas,State_United States,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,2017,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,2017,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2017,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2017,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2017,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# Define the target data set.
y = death_df["Death Rate"].ravel()
y[:5]

array([49.4, 53.8, 63.7, 56.2, 51.8])

In [15]:
# Split the dataset into training and testing set (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)


In [16]:
# Define the Estimator and the Parameter Ranges
dt = RandomForestRegressor()
number_of_iterations = 20
max_leaf_nodes = [2, 3, 4, 5, 6, 7]
min_samples_split = [5, 10, 20, 50]
max_depth = [5,10,15,20]
max_features = [3,4,5]
n_estimators = [50, 100, 200]

# Define the param distribution dictionary
param_distributions = dict(max_leaf_nodes=max_leaf_nodes, 
                           min_samples_split=min_samples_split, 
                           max_depth=max_depth,
                           max_features=max_features,
                           n_estimators=n_estimators)

# Build the gridsearch
grid = RandomizedSearchCV(estimator=dt, 
                          param_distributions=param_distributions, 
                          n_iter=number_of_iterations, 
                          cv = 5)

grid_results = grid.fit(x_train, y_train)

# Summarize the results in a readable format
print("Best params: {0}, using {1}".format(grid_results.cv_results_['mean_test_score'], grid_results.best_params_))
results_df = pd.DataFrame(grid_results.cv_results_)

NameError: name 'RandomizedSearchCV' is not defined

In [ ]:
# Initializing the Random Forest Regression model with 128 decision trees
model = RandomForestRegressor(n_estimators =128, max_features = 'sqrt', max_depth = 7, random_state = 0)


In [ ]:
# Fitting the Random Forest Regression model to the data
model.fit(X_train, y_train)

In [ ]:
# Predicting the target values of the test set
y_pred = model.predict(X_test)

To calculate the loss between the actual target values in the testing set and the values predicted by the model, a cost function called the Root Mean Square Error (RMSE) is used.

The RMSE  indicates how close the actual data points are to the model’s predicted values. When RMSE value is low it indicates a better fit and is a good measure for determining the accuracy of the model’s predictions.

In [ ]:
# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)),'.3f'))
print("\nRMSE:\n",rmse)

# RMSE for training....verifying if data leakage was occuring
#rmse = float(format(np.sqrt(mean_squared_error(y_train, model.predict(X_train))),'.3f'))
#print("\nRMSE:\n",rmse)

R² score indicates how will our model is fitte to the data by comparing it to the average line of the dependent variable (y or target). A score closer to 1, implies our model preformed well verses if score is further from 1, implying our model did not perform well

In [ ]:
# R² score.....indicates that 20% of the variancce in the causes of death can be explained??? 
r2_score(y_test, y_pred)

In [ ]:
model.feature_importances_

In [ ]:
df_feature_importance = pd.DataFrame({'feature': X.columns, 'feature_importance':model.feature_importances_})
df_feature_importance.head()

In [ ]:
df_feature_importance.sort_values('feature_importance')